In [ ]:

!pip install opencv-python
!pip install face_recognition
!pip install moviepy
!pip install dlib==19.18.0
!pip install face_recognition

In [ ]:

from google.colab import files
uploaded = files.upload()

In [ ]:


import cv2
import os

def extract_frames_from_videos(video_paths, output_folder_prefix):
    for idx, video_path in enumerate(video_paths):
        output_folder = f"{output_folder_prefix}_{idx}"
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        video_capture = cv2.VideoCapture(video_path)
        success, image = video_capture.read()
        count = 0

        while success:
            cv2.imwrite(os.path.join(output_folder, f"{count}.jpg"), image)
            success, image = video_capture.read()
            count += 1

# Daftar video yang telah diunggah
video_files = ['class1.mp4', 'class2.mp4', 'class3.mp4']

# Menentukan folder untuk menyimpan frame
output_folder_prefix = 'nama siswa'
extract_frames_from_videos(video_files, output_folder_prefix)

In [ ]:


import face_recognition
import matplotlib.pyplot as plt
import os

def detect_faces_in_multiple_folders(folders):
    all_face_locations = []
    for folder in folders:
        face_locations = []
        frame_files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

        for frame_file in frame_files:
            frame_path = os.path.join(folder, frame_file)
            image = face_recognition.load_image_file(frame_path)
            face_locations_in_frame = face_recognition.face_locations(image, model="hog")  # Gunakan HOG

            for face_location in face_locations_in_frame:
                face_locations.append((frame_path, face_location))

        all_face_locations.append(face_locations)
    return all_face_locations

# Daftar folder hasil ekstraksi frame dari setiap video
folders = [f"video_frames_{i}" for i in range(len(video_files))]

# Mendeteksi wajah dalam semua folder
all_face_locations = detect_faces_in_multiple_folders(folders)
print(f"Detected faces in {sum(len(locations) for locations in all_face_locations)} frames.")


In [ ]:


def show_faces(face_locations):
    for frame_path, face_location in face_locations:
        image = cv2.imread(frame_path)
        top, right, bottom, left = face_location
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.show()

# Daftar nama file video yang diunggah
video_files = ['class1.mp4', 'class2.mp4', 'class3.mp4']

# Daftar folder hasil ekstraksi frame dari setiap video
folders = [f"video_frames_{i}" for i in range(len(video_files))]

# Mendeteksi wajah dalam semua folder
all_face_locations = detect_faces_in_multiple_folders(folders)
print(f"Detected faces in {sum(len(locations) for locations in all_face_locations)} frames.")

# Gabungkan semua deteksi wajah menjadi satu daftar
combined_face_locations = [item for sublist in all_face_locations for item in sublist]

# Tampilkan deteksi wajah
show_faces(combined_face_locations)

In [ ]:

import os

def save_extracted_faces(face_locations, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for idx, (frame_path, face_location) in enumerate(face_locations):
        image = cv2.imread(frame_path)
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        face_filename = os.path.join(output_folder, f"face_{idx}.jpg")
        cv2.imwrite(face_filename, face_image)

# Folder untuk menyimpan wajah yang diekstrak
output_faces_folder = 'extracted_faces'

# Simpan wajah yang diekstrak
save_extracted_faces(combined_face_locations, output_faces_folder)
print(f"Faces saved to {output_faces_folder}")


Faces saved to extracted_faces


In [ ]:
import shutil

# Membuat file zip dari folder extracted_faces
shutil.make_archive(output_faces_folder, 'zip', output_faces_folder)

# Mengunduh file zip
from google.colab import files
files.download(f'{output_faces_folder}.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import cv2  # Import modul cv2

def load_data(folder_path, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        label = filename.split("_")[0]  # Ambil label dari nama file (misal: 'class1.jpg' -> 'class1')
        image = cv2.imread(img_path)
        resized_image = cv2.resize(image, target_size)  # Mengubah ukuran gambar
        images.append(resized_image)
        labels.append(label)
    return np.array(images), np.array(labels)

# Folder tempat menyimpan wajah yang diekstrak
extracted_faces_folder = 'extracted_faces'

# Muat data untuk pelatihan
images, labels = load_data(extracted_faces_folder)

# Bagi data menjadi data latih dan data uji
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# Fungsi untuk membuat model
def create_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Muat data untuk pelatihan
images, labels = load_data(extracted_faces_folder)

# Bagi data menjadi data latih dan data uji
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Ukuran gambar wajah (misal: 100x100 piksel) dan jumlah kelas
input_shape = (100, 100, 3)
num_classes = len(set(labels))

# Buat model
model = create_model(input_shape, num_classes)

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Buat objek LabelEncoder
label_encoder = LabelEncoder()

# Konversi label menjadi tipe data numerik
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Compile dan latih model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels_encoded, epochs=10, validation_data=(test_images, test_labels_encoded))

In [ ]:
loss, accuracy = model.evaluate(test_images, test_labels_encoded)
print("Loss on test data:", loss)
print("Accuracy on test data:", accuracy)


In [ ]:
import cv2
import numpy as np

# Paths to the uploaded video files
video_paths = ['class1.mp4', 'class2.mp4', 'class3.mp4']  # Ganti dengan nama file video Anda

# Fungsi untuk memproses gambar
def preprocess_image(image):
    # Normalisasi gambar
    image = image / 255.0
    return image

# Loop over each video
for video_path in video_paths:
    # Buka video
    video = cv2.VideoCapture(video_path)

    # Periksa apakah video berhasil dibuka
    if not video.isOpened():
        print(f"Gagal membuka video {video_path}. Periksa kembali jalur file.")
    else:
        print(f"Video {video_path} berhasil dibuka.")

        # Ambil FPS (Frame Per Second) dari video
        fps = int(video.get(cv2.CAP_PROP_FPS))

        # Loop over frames in the video
        while True:
            # Ekstrak frame
            ret, frame = video.read()

            if not ret:
                print(f"Video {video_path} selesai diproses.")
                break

            # Resize frame to match input shape of the model
            resized_frame = cv2.resize(frame, (100, 100))  # Sesuaikan dengan input model

            # Preprocess frame (e.g., normalization)
            preprocessed_frame = preprocess_image(resized_frame)

            # Convert frame to array and add batch dimension
            input_data = np.expand_dims(preprocessed_frame, axis=0)

            # Use model for prediction
            predictions = model.predict(input_data)
            print(f"Predictions for frame in {video_path}:", predictions)

            # Jika Anda ingin memberhentikan pemrosesan setelah beberapa frame, tambahkan kondisi di sini
            # Misalnya, jika Anda ingin menghentikan pemrosesan setelah 100 frame, Anda bisa tambahkan:
            # if frame_count >= 100:
            #     break

        # Jangan lupa untuk melepaskan objek video
        video.release()


In [ ]:
import matplotlib.pyplot as plt

# Prediksi model
predictions = [0.2, 0.5, 0.3]  # Contoh hasil prediksi, ganti dengan hasil prediksi sesuai kebutuhan

# Labels kelas
class_labels = ['Class 1', 'Class 2', 'Class 3']  # Ganti dengan label kelas Anda

# Visualisasi hasil prediksi
plt.figure(figsize=(8, 6))
plt.bar(class_labels, predictions, color='blue')
plt.xlabel('Classes')
plt.ylabel('Predicted Probabilities')
plt.title('Predictions')
plt.ylim(0, 1)  # Rentang sumbu y dari 0 hingga 1
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [ ]:
import pandas as pd

# Contoh hasil prediksi dan label kelas
predictions = [[0.2, 0.5, 0.3], [0.8, 0.1, 0.1], [0.4, 0.4, 0.2]]  # Contoh hasil prediksi, ganti dengan hasil prediksi sesuai kebutuhan
class_labels = ['Class 1', 'Class 2', 'Class 3']  # Ganti dengan label kelas Anda

# Membuat dataframe dari hasil prediksi
df_predictions = pd.DataFrame(predictions, columns=class_labels)

# Menyimpan dataframe ke dalam file CSV
csv_file_path = 'predictions.csv'  # Ganti dengan jalur file yang Anda inginkan
df_predictions.to_csv(csv_file_path, index=False)

print("Hasil prediksi telah disimpan dalam file CSV:", csv_file_path)


In [ ]:
from google.colab import files

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>